![IE](../img/ie.png)

# Sessions 9 & 10: Multi-process and asynchronous programming

### Juan Luis Cano Rodríguez <jcano@faculty.ie.edu> - Master in Business Analytics and Big Data (2019-04-24)

## First rule of performance analysis

> "We should forget about small efficiencies, say about 97% of the time: premature optimization is the root of all evil."
>
> — Donald E. Knuth

> **premature** (adj.) pre·ma·ture : happening, arriving, existing, or performed before the proper, usual, or intended time
>
> Merriam-Webster dictionary

There are several techniques to make slow programs go faster. However, no matter what we do, the first step is to **analyze** _where does the slowdown come from_.

Among the most straight-forward tools to analyze performance in Python are cProfile and line_profiler, and they are very easy to use from the notebook. cProfile is part of the standard library, whereas line_profiler must be installed with pip:

In [1]:
#!pip install line_profiler  # Python <= 3.6
!pip install cython  # Required to install line_profiler from source
!pip install https://github.com/rkern/line_profiler/archive/master.zip  # Required in Python 3.7

     / 245kB 716kB/s
  Running setup.py bdist_wheel for line-profiler ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-qra37gu9/wheels/c0/45/5c/273eebba2150941af640a9801c65dec439f35f747e694aeb52
Successfully built line-profiler
